In [1]:
pip install --upgrade scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import the libraries

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
# load data set 

data = pd.read_csv('insurance.csv')

In [4]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
x = data.drop(columns=['charges'])
y = data['charges']

In [6]:
x.head()

,age,sex,bmi,children,smoker,region
0,19,female,27.900,0,yes,southwest
1,18,male,33.770,1,no,southeast
2,28,male,33.000,3,no,southeast
3,33,male,22.705,0,no,northwest
4,32,male,28.880,0,no,northwest


In [7]:
# encode the categorical variable 

label_encoders = {}
cat_cols = list(x.select_dtypes(include='object').columns)
for col in cat_cols:
    encoder = LabelEncoder() # init the encoder
    encoder.fit(x[col])
    x[col] = encoder.transform(x[col])
    label_encoders[col] = encoder

In [8]:
x.head()

,age,sex,bmi,children,smoker,region
0,19,0,27.900,0,1,3
1,18,1,33.770,1,0,2
2,28,1,33.000,3,0,2
3,33,1,22.705,0,0,1
4,32,1,28.880,0,0,1


In [9]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.15,
                                                    random_state=24)

for output in [x_train, x_test, y_train, y_test]:
    print(output.shape)

(1137, 6)
(201, 6)
(1137,)
(201,)


### Scale the dataset

In [10]:
# init the scaler
scaler = StandardScaler()
column_names = list(x.columns)
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_train = pd.DataFrame(data = x_train, columns=column_names)
x_test = pd.DataFrame(data=x_test, columns=column_names)


### Train the model

In [11]:
model = LinearRegression() #init the model
model.fit(x_train, y_train) #fit the model into the data
train_preds = model.predict(x_train) # get training prediction
test_preds = model.predict(x_test) # get the prediction 


In [12]:
train_rmse = root_mean_squared_error(y_true=y_train, y_pred=train_preds)
test_rmse = root_mean_squared_error(y_true=y_test, y_pred=test_preds)

print(f'train_rmse: {train_rmse}')
print(f'test_rmse: {test_rmse}')


train_rmse: 6064.086697828254
test_rmse: 210078.60430775204


### MAKING INFERENCE.

In [ ]:
def predict_charges (**inputs) ->pd.DataFrame:
    inputs = {name: [value] for name, value in inputs.items()}
    client_data = pd.DataFrame(data=inputs)
    for col in cat_cols:
        client_data[col] = label_encoders[col].transform(client_data[col])
    client_data = scaler.transform(client_data)
    client_data = pd.DataFrame(data = client_data, columns=column_names)
    prediction = model.predict(client_data)

    return f'This customer is expected to be pay ${round(prediction[0], 3)}'

predict_charges(age = 43, sex = 'female', bmi = 35.5, children = 2, smoker = 'no', region = 'southwest')

'This customer is expected to be $ 11084.67'